In [ ]:
#LSTM caso 1: UNIVARIADO - CANTIDAD_VENTAS
import pyodbc
import pandas as pd
from datetime import date, datetime, timedelta
import numpy as np
from dateutil.relativedelta import relativedelta
import matplotlib.pylab as plt
%matplotlib inline
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, LSTM
from keras.optimizers import SGD, RMSprop
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest

plt.rcParams['figure.figsize' ] = (16, 9)
plt.style.use('fast')

In [ ]:
sql_server = """
DRIVER={ODBC Driver 17 for SQL Server};
server=192.168.1.235,1433;
database=demo_prediccion;
uid=sa;
pwd=qwerty;
Trusted_connection=yes;
"""

In [ ]:
# Se entrenará con todos los datos de las ventas menores al 2020
query = """
SELECT
[F].[Fecha] AS fecha,
SUM([H]. [cantidad]) AS TotalVentas
FROM
[demo_prediccion]. [dbo]. [hechos] AS [H]
INNER JOIN [demo_prediccion]. [dbo]. [Dim_fechas] AS [f] ON [H].[id_DimFechas] = [F].[id]
WHERE 
[F].[Fecha] > '2016-12-31' AND [F].[Fecha] < '2020-01-01'
GROUP BY [F].[Fecha]
ORDER BY [F].[Fecha]
"""

In [ ]:
query2="""
SELECT
[F].[Fecha] AS fecha,
SUM([H]. [cantidad]) AS TotalVentas
FROM
[demo_prediccion]. [dbo]. [hechos] AS [H]
INNER JOIN [demo_prediccion]. [dbo]. [Dim_fechas] AS [f] ON [H].[id_DimFechas] = [F].[id]
WHERE 
[F].[Fecha] > '2016-12-31' AND [F].[Fecha] < '2021-01-01'
GROUP BY [F].[Fecha]
ORDER BY [F].[Fecha]
"""

In [ ]:
PASOS = 31

In [ ]:
#Opcion 1: Permitir que pueda ser modificada la sentencia, y mantener el try para evitar de que
# el programa se rompa
def get_sqlconnection(config_sqlServer):
    status = "inicializando...."
    try: 
        connection = pyodbc.connect(sql_server)
        status = "Conexion establecida satisfactoriamente"
    except Exception as e:
        status = "Error al establecer la conexión:"+e
    print(status)
    return connection

In [ ]:
def set_index_datetime(data):
        if str(type(data) == "<class 'pandas.core.frame.DataFrame'>"):
            # data.sort_values('fecha', inplace=True)
            for column in data.columns: 
                try: 
                    pd.to_datetime(data[column])
                    data.set_index(column,inplace=True)
                    return data
                except Exception as e:  
                    pass
        else: 
            return 0

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
def eliminar_anomalias(dtaframe):
    dataFrame_anomalias = dtaframe.copy()
    modeloIsolation = IsolationForest(contamination=0.05)
    modeloIsolation.fit(dataFrame_anomalias)
    anomalias = modeloIsolation.predict(dataFrame_anomalias)
    dtaframe['anomalias' ] = anomalias
    dataFrameSinAnomalias = dtaframe[dtaframe['anomalias' ] != -1]
    dataFrameSinAnomalias = dataFrameSinAnomalias.drop('anomalias', axis=1)
    return dataFrameSinAnomalias

In [ ]:
#core
with get_sqlconnection(sql_server) as cursor: 
    prepData = pd.read_sql_query(query,cursor)
    historyData = pd.read_sql_query(query2, cursor)

    prepData = set_index_datetime(prepData)
    historyData = set_index_datetime(historyData)

    #Switch the format on days or months
    #nota, acualmente los datos son filtrados con respecto a solo si son en  formato de día o de mes,
    #en LSTM, que incluye  datos en horarios, se tendría que modificar esta parte
    try: 
        #Si se trata de meses
        first_day = datetime.strptime(prepData.index.min(),'%Y-%m') + relativedelta(months=1)
        last_day = datetime.strptime(prepData.index.max(), '%Y-%m' ) + relativedelta(months=1)
        future_days = [last_day + relativedelta(months=i) for i in range(PASOS)]
        for i in range(len(future_days)):
            future_days[i] = str(future_days[i])[:7]
        print("Month_format_detected")
    except Exception as e: 
        #si se trata de dias
        print("Days_format_detected")
        first_day = prepData.index.min() + timedelta(days=1)
        last_day = prepData.index.max() + timedelta(days=1)
        future_days = [last_day + timedelta(days=i) for i in range(PASOS)]
        for i in range(len(future_days)):
            future_days[i] = str(future_days[i])[:10]
    #END TRY

    #De momento solamente será la fecha
    future_data = pd.DataFrame(future_days, columns=['fecha'])

    
    #creamos un modelo
    for column in prepData.columns:
        data = prepData.filter([column])
        data = prepData.filter([column])
        data.set_index(prepData.index, inplace=True)
        data = eliminar_anomalias(data)